In [2]:
import re
import nltk

In [3]:
import re, string
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.tokenize import WhitespaceTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim import corpora
from gensim.models.ldamodel import LdaModel
from collections import Counter
#import pickle
#from gensim.test.utils import datapath

In [4]:
# Create a WordNetLemmatizer object
lemmatizer = WordNetLemmatizer()

In [5]:
# read lines as lists
lines_each = []
with open("ratemd.25k.all.txt", "r") as f:
    lines_each.append(f.readlines())

# Task#1: Corpus collection and Corpus Descriptive analysis 

## Problem#1:
Do a descriptive analysis of your corpus and provide (in the table below): the distribution of reviews per gender and sentiment (show both count and percent coverage). 

Here the sentiment can be only positive or negative -- determined by mapping the overall ratings of at most 3 into negative (i.e., [1,3]) and those at least 4 into positive (i.e., [4,5]). E.g., the overall rating of the example above maps into positive sentiment.

In [8]:
# create English stop words list (you can always define your own stopwords)
stop_words = set(stopwords.words('english'))

In [9]:
# Function to remove stop words from sentences & lemmatize verbs and nouns. 
def clean(doc, lem = True):
    tokenized = word_tokenize(doc.lower())
    stop_free = [x for x in tokenized if not re.fullmatch('[' + string.punctuation + ']+', x) and x not in stop_words]
    if lem:
        lemma_verb = [lemmatizer.lemmatize(word,'v') for word in stop_free]
        lemma_noun = [lemmatizer.lemmatize(word,'n') for word in lemma_verb]
        #y = [s for s in lemma_noun if len(s) > 2]
        return lemma_noun
    else:
        return stop_free

In [13]:
lines_lower = [x.lower() for x in lines_each[0]]

In [14]:
doc_name = re.compile(r"dr.\s\w*\s\w*[.]?\s?\w*\s\w*\s?\t\s[a-z]{0,2}male")

In [15]:
gender = {}
rating = {}
doc = 0
for x in lines_lower:
    m = re.search(doc_name, x)
    if m:
        doc += 1 #=x[:m.end()-4]
    search = re.findall(r'(male|female)', x)
    if len(search) > 0:
        gender[doc] = search[0]
    match = re.search('overall rating:\s\d.\d+', x)
    if match:
        if match.end() > 19:
            new = x[match.end()-4:match.end()]
        else:
            new = x[match.end()-3:match.end()]
        try:
            old = rating[doc]
            rep = old + "\s" + new
            rating[doc] = rep
        except:
            rating[doc] = new

In [17]:
# there are doctors without ratings
dif = list(set(gender.keys()) - set(rating.keys()))

In [18]:
f_rating = []
m_rating = []
m = 0
f = 0

for k, v in gender.items():
    if k not in dif:
        score = rating[k].split(r"\s")
        for s in score:
            if v == "male":
                m_rating.append(float(s))
                m += 1
            else:
                f_rating.append(float(s))
                f += 1
    else:
        if v == "male":
            m_rating.append(None)
            m += 1
        else:
            f_rating.append(None)
            f += 1

In [20]:
f_p = 0
f_n = 0
for x in f_rating:
    if x:
        if x >= 4:
            f_p += 1
        elif x>= 0:
            f_n += 1
m_p = 0
m_n = 0
for x in m_rating:
    if x:
        if x >= 4:
            m_p += 1
        elif x>= 0:
            m_n += 1

In [21]:
tbl = (f_p, f_n, f_p/len(f_rating)*100, 
       f_n/len(f_rating)*100,len(f_rating),
       len(f_rating)/(m+f)*100,
       m_p, m_n, 
       m_p/len(m_rating)*100, 
       m_n/len(m_rating)*100,
       len(m_rating),
       len(m_rating)/(m+f)*100)

In [22]:
P1_tbl = pd.DataFrame(np.array(np.reshape(tbl, (2,6))))

In [23]:
# give table index and column
P1_tbl.columns = ["Positive_cnt", "Negative_cnt",
                 "Positive_%", "Negative_%","Total_cnt", 
                  "Total_%"]
P1_tbl['Gender'] = ['Female', 'Male']
P1_tbl = P1_tbl.set_index("Gender")

In [24]:
P1_tbl

,Positive_cnt,Negative_cnt,Positive_%,Negative_%,Total_cnt,Total_%
Gender,,,,,,
Female,3030.0,2263.0,56.372093,42.102326,5375.0,25.88241
Male,9532.0,5595.0,61.928274,36.350052,15392.0,74.11759


In [25]:
with open('analysis.txt', 'w+') as f:
    f.writelines("\nTask 1 Problem 1\n")

In [26]:
P1_tbl.to_csv(r'analysis.txt', sep=' ', mode='a')

Also provide and comment on the size of the reviews in the corpus: i.e., the length of the smallest review and of the largest review, as well as the average length of the reviews in the corpus. 

Here the length of a review is defined as the number of raw tokens (i.e., any sequence of characters separated by space and/or beginning/end of review).

In [27]:
review = []
for x in lines_lower:
    match = re.search(r'overall rating:\s\d.\d+\s\t\s', x)
    if match:
        review.append(x[match.end():-1])

In [28]:
score = []
for x in lines_lower:
    match = re.search(r'overall rating:\s\d.\d+\s\t\s', x)
    if match:
        if match.end() > 22:
            score.append(float(x[match.end()-7:match.end()-3]))
        else:
            score.append(float(x[match.end()-6:match.end()-3]))

In [29]:
ws_tokenize = WhitespaceTokenizer()

In [30]:
tok_review = []
for r in review:
    if len(ws_tokenize.tokenize(r)) > 0:
        tok_review.append(len(ws_tokenize.tokenize(r)))

In [31]:
P2_tbl = pd.DataFrame(tok_review).describe().T[['min','mean', 'max']]

In [99]:
P2_tbl

,min,mean,max
0,1.0,65.360652,899.0


In [32]:
with open('analysis.txt', 'a') as f:
    f.writelines("\nTask 1 Problem 1 EDA\n")

In [33]:
P2_tbl.to_csv(r'analysis.txt', index=None, sep=' ', mode='a')

## Problem#2:
Why is this dataset from RateMD a valid, relevant corpus for your project?

For this, you are referred to the corpus design principles discussed in class (Lecture 5). 

In particular, consider the following helping questions and fill in the entries in the table below.

In [35]:
P12_tbl = pd.DataFrame(score).describe().T
P12_tbl

,count,mean,std,min,25%,50%,75%,max
0,20420.0,3.741467,1.480644,0.75,2.25,4.5,5.0,5.0


In [36]:
P22_tbl = pd.DataFrame()

In [37]:
P22_tbl['Questions'] = ['What is the language variety of the corpus (i.e., genre)?',
'What is the size of the corpus?',
'What meta-data is provided with the reviews?',
'What socio-demographic information is provided about the patients who wrote the reviews?',
'Is the corpus balanced along the meta-data dimensions considered? (look only at sentiment and gender)']

In [38]:
P22_tbl['RateMD corpus'] = ['Reviews written by anyone who has access to the internet',
'19,645 non-empty reviews',
'Doctor’s name, gender, clinic location, specialty, rating, review',
'Nothing',
'No. 1) There are significantly more male doctors being reviewed. 2) The ratings are not uniform. There are significantly more score 5 reviews (see table below).']

In [39]:
P22_tbl['Healthcare company’s corpus'] =['Reviews written by patients of the company’s clinics',
'500,000 reviews',
'Doctor’s name, gender, clinic location; review sentiment',
'Gender, age, economic and educational status',
'No (the dimensions are not uniformly distributed; they exhibit a natural distribution)']

In [40]:
with open('analysis.txt', 'a') as f:
    f.writelines("\nTask 1 Problem 2\n")

In [41]:
P22_tbl.to_csv(r'analysis.txt', sep=' ', mode='a')

In [42]:
P12_tbl.to_csv(r'analysis.txt', index=None, sep=' ', mode='a')

# Task#2: Exploratory Analysis of Corpus with LDA

You have to write a python program that takes as input the corpus (i.e., your RateMD corpus), a given number of topics k, and generates these topics. For this task you will experiment with LDA (Latent Dirichlet Allocation).


### Step 1: Clean the corpus
First, by looking at the stats you got on the corpus in Problem1, you realize that some comments are empty or contain just one word. Thus, here are some possible cleaning procedures you might want to do:

- convert the text reviews to lowercase 
- remove reviews with less than 3 tokens

In [43]:
long_review = [x for x in review if len(x) > 3]

- tokenize them 
- remove punctuations, and stop words.
- You also decide to filter the terms which occurred less than 10 times in your documents (i.e., reviews).
- with and without lemmatization

In [44]:
# with lemmatization
skim_review = [clean(doc.strip()) for doc in long_review]

In [45]:
# without lemmatization
ori_review = [clean(doc.strip(), False) for doc in long_review]

### Step 2: Create the dictionary
After you cleaned your corpus, you will create the term dictionary. How large is your dictionary?

-With lemmatization

In [46]:
from collections import Counter

In [47]:
freq = [(k,v) for k,v in Counter([item for sublist in skim_review for item in sublist]).items() 
        if v >= 10 and re.search('\d+', k) is None]

In [48]:
dictionary = [x[0] for x in freq]

-Without lemmatization

In [50]:
ori_freq = [(k,v) for k,v in Counter([item for sublist in ori_review for item in sublist]).items() 
            if v >= 10 and re.search('\d+', k) is None]

In [51]:
ori_dictionary = [x[0] for x in ori_freq]

In [53]:
with open('analysis.txt', 'a') as f:
    f.writeline('\nTask 2\n')
    f.writelines('Length of dictionary without lemmatization is {}.\n'.format(len(ori_dictionary)))
    f.writelines('Length of dictionary with lemmatization is {}.\n'.format(len(dictionary)))

### Step 3: 
Convert the list of documents in your corpus into Document-Term Matrix using the dictionary prepared at Step 2 (again, a term is a word).

-With lemmatization

In [54]:
Dic = corpora.Dictionary([dictionary])

In [55]:
corpus = [Dic.doc2bow(doc_clean) for doc_clean in skim_review]

-Without lemmatization

In [56]:
ori_Dic = corpora.Dictionary([ori_dictionary])

In [57]:
ori_corpus = [Dic.doc2bow(doc_clean) for doc_clean in ori_review]

### Step 4: Run the LDA model on the document-term matrix

Here you have to run LDA with the following parameters: number of topics (k = 10), number of passes (pass = 20), and number of iterations (iterations = 2000).

-With lemmatization

In [58]:
# LDA model needs many iterations/passes and a large corpus to work well
# must define the number of topics you want to extract from the corpus
ldamodel = LdaModel(corpus, num_topics=10, id2word = Dic, passes=20, iterations=2000)

-Without lemmatization

In [59]:
# LDA model needs many iterations/passes and a large corpus to work well
# must define the number of topics you want to extract from the corpus
ori_ldamodel = LdaModel(ori_corpus, num_topics=10, id2word = ori_Dic, passes=20, iterations=2000)

### Step 5: For each of the k topics, print the top 10 words

## Problem#1: Without lemmatization

In [60]:
# print out the top 10 words in each topic
ori_tops = ori_ldamodel.print_topics(num_topics=10, num_words=10)

In [61]:
tbl_21_ori = pd.DataFrame([re.findall("\"\w*[.']?\w*\"", x[1]) for x in ori_tops[:5]])

In [62]:
tbl_21_ori['topic'] = ['Specific Cases', 'Professional Doctors', 'Thorough Treatment', 'Lecturing Doctors', 'Bad Experience']

In [63]:
tbl_21_ori

,0,1,2,3,4,5,6,7,8,9,topic
0,"""cut""","""data""","""'s""","""baldwin""","""dipatri""","""complaint""","""expected""","""bye""","""dascombe""","""exam""",Specific Cases
1,"""nami""","""data""","""cut""","""service""","""precious""","""discovered""","""figured""","""imagine""","""expected""","""hateful""",Professional Doctors
2,"""glazer""","""data""","""individual""","""loewe""","""bothered""","""father""","""'s""","""main""","""cheap""","""induction""",Thorough Treatment
3,"""bonakdar""","""data""","""refreshing""","""physican""","""home""","""ford""","""biopsy""","""professional""","""reply""","""contacted""",Lecturing Doctors
4,"""jensen""","""cut""","""service""","""everyone""","""kent""","""honor""","""ethics""","""diligent""","""closely""","""paralyzed""",Bad Experience


In [64]:
tbl_22_ori = pd.DataFrame([re.findall("\"\w*[.']?\w*\"", x[1]) for x in ori_tops[5:]])

In [65]:
tbl_22_ori['topic'] = ['Neutral Visit', 'Retired Patient?', 'Emotional Visit?', 'Not Great Experience', 'Saturday Light-Hearted Visits']

In [66]:
tbl_22_ori

,0,1,2,3,4,5,6,7,8,9,topic
0,"""laptop""","""precious""","""redo""","""sarcastic""","""anterior""","""ethics""","""blown""","""paralyzed""","""data""","""cut""",Neutral Visit
1,"""data""","""relaxed""","""seeing""","""lazy""","""mexico""","""overall""","""dascombe""","""knowing""","""panchal""","""hemorrhoids""",Retired Patient?
2,"""data""","""redo""","""alcohol""","""drink""","""cut""","""precious""","""honor""","""expected""","""much""","""'s""",Emotional Visit?
3,"""data""","""pulmonary""","""mian""","""service""","""pulling""","""nami""","""easier""","""medicine""","""dislike""","""associated""",Not Great Experience
4,"""likely""","""pulmonary""","""data""","""associates""","""dascombe""","""home""","""realized""","""closely""","""everytime""","""hart""",Saturday Light-Hearted Visits


> All of the topics without lemmatization looks confusing.

## Problem#2: With lemmatization

In [67]:
# print out the top 10 words in each topic
tops = ldamodel.print_topics(num_topics=10, num_words=10)

In [68]:
tbl_21 = pd.DataFrame([re.findall("\"\w*[.']?\w*\"", x[1]) for x in tops[:5]])

In [69]:
tbl_21['topic'] = ['Bad Experience', 'High Cost', 'Baby Delivery Doctor', 'Great Review', 'Long Wait Time']

In [70]:
tbl_21

,0,1,2,3,4,5,6,7,8,9,topic
0,"""dr.""","""staff""","""care""","""doctor""","""recommend""","""great""","""would""","""knowledgeable""","""highly""","""patient""",Bad Experience
1,"""surgery""","""dr.""","""procedure""","""surgeon""","""result""","""would""","""recommend""","""perform""","""go""","""breast""",High Cost
2,"""manner""","""bedside""","""doctor""","""good""","""great""","""life""","""save""","""best""","""side""","""excellent""",Baby Delivery Doctor
3,"""doctor""","""patient""","""dr.""","""time""","""care""","""'s""","""n't""","""like""","""take""","""know""",Great Review
4,"""office""","""wait""","""doctor""","""staff""","""time""","""n't""","""get""","""rude""","""appointment""","""see""",Long Wait Time


In [71]:
tbl_22 = pd.DataFrame([re.findall("\"\w*[.']?\w*\"", x[1]) for x in tops[5:]])

In [72]:
tbl_22['topic'] = ['Neutral Visit', 'Attentive Doctors', 'Got Surgery', 'Common Visit', 'Good Doctor for All']

In [73]:
tbl_22

,0,1,2,3,4,5,6,7,8,9,topic
0,"""dr.""","""child""","""u""","""son""","""make""","""daughter""","""baby""","""'s""","""feel""","""love""",Neutral Visit
1,"""pain""","""surgery""","""dr.""","""year""","""back""","""life""","""dr""","""knee""","""month""","""would""",Attentive Doctors
2,"""call""","""dr.""","""patient""","""medical""","""test""","""insurance""","""result""","""office""","""get""","""doctor""",Got Surgery
3,"""question""","""answer""","""time""","""ask""","""take""","""dr.""","""rush""","""make""","""concern""","""appointment""",Common Visit
4,"""n't""","""go""","""tell""","""say""","""get""","""would""","""doctor""","""see""","""could""","""ask""",Good Doctor for All


> Topic 6 and 8 are hard to label

## Problem#3:
Compare your program’s output with and without lemmatization. Which of these settings generates better topics? Is lemmatization worth doing? (compare the goodness of the topics with and without lemmatization). Explain.


> Lemmatized documents generated better topics. Each topic has a common thread and different topics are quite divergent from each other. Thus, it is worth it to lemmatize.

In [74]:
with open('analysis.txt', 'a') as f:
    f.writelines("\nTask 2 Problem 1: Topics without lemmatization\n")

In [75]:
tbl_21_ori.to_csv(r'analysis.txt', index=None, sep=' ', mode='a')
tbl_22_ori.to_csv(r'analysis.txt', index=None, sep=' ', mode='a')

In [76]:
with open('analysis.txt', 'a') as f:
    f.writelines('All of the topics without lemmatization looks confusing.\n')
    f.writelines("\nTask 2 Problem 2: Topics with lemmatization\n")

In [77]:
tbl_21.to_csv(r'analysis.txt', index=None, sep=' ', mode='a')
tbl_22.to_csv(r'analysis.txt', index=None, sep=' ', mode='a')

In [78]:
with open('analysis.txt', 'a') as f:
    f.writelines("Only topic 6 and 8 are hard to label.\n")
    f.writelines("\nTask 2 Problem 3\n")
    f.writelines("Lemmatized documents generated better topics. Each topic has a common thread and different topics are quite divergent from each other. Thus, it is worth it to lemmatize.\n")

# Extra-credit

## Problem#1: [10 points]
Repeat Task#2 but this time with k = 20 topics.
What do you notice? Are the results with 20 topics better than those with 10 topics (under both with and without lemmatization scenarios)? Again, ‘better results’ here refers to the goodness of your topics.


-With lemmatization

In [83]:
# LDA model needs many iterations/passes and a large corpus to work well
# must define the number of topics you want to extract from the corpus
ldamodel_2 = LdaModel(corpus, num_topics=20, id2word = Dic, passes=20, iterations=2000)

-Without lemmatization

In [84]:
# LDA model needs many iterations/passes and a large corpus to work well
# must define the number of topics you want to extract from the corpus
ori_ldamodel_2 = LdaModel(ori_corpus, num_topics=20, id2word = ori_Dic, passes=20, iterations=2000)

###  - Without lemmatization

In [85]:
# print out the top 10 words in each topic
ori_tops_2 = ori_ldamodel_2.print_topics(num_topics=20, num_words=10)

In [87]:
tbl_ec1_ori = pd.DataFrame([re.findall("\"\w*[.']?\w*\"", x[1]) for x in ori_tops_2])

In [88]:
tbl_ec1_ori

,0,1,2,3,4,5,6,7,8,9
0,"""data""","""point""","""'s""","""cheap""","""home""","""individual""","""feel""","""immensely""","""seeing""","""pa"""
1,"""pulling""","""physican""","""bonakdar""","""data""","""baldwin""","""everytime""","""manager""","""emch""","""hr""","""refreshing"""
2,"""service""","""associates""","""jensen""","""customer""","""bigger""","""diligent""","""everyone""","""kent""","""laughing""","""hr"""
3,"""data""","""nami""","""cut""","""baldwin""","""service""","""figured""","""anal""","""discovered""","""loewe""","""bothered"""
4,"""data""","""pulmonary""","""ford""","""mian""","""home""","""nasri""","""realized""","""service""","""easier""","""completed"""
5,"""lazy""","""becoming""","""copies""","""several""","""data""","""cut""","""panchal""","""ready""","""mexico""","""amazingly"""
6,"""glazer""","""laptop""","""cut""","""loewe""","""bothered""","""individual""","""service""","""looked""","""iron""","""past"""
7,"""docs""","""assist""","""decent""","""recommending""","""disc""","""reply""","""corrective""","""determined""","""calming""","""fractured"""
8,"""data""","""drink""","""redo""","""alcohol""","""precious""","""honor""","""caution""","""expected""","""ignored""","""humiliated"""
9,"""cut""","""operations""","""precious""","""dipatri""","""kent""","""series""","""service""","""laptop""","""endocrinologist""","""everyone"""


> All of the topics without lemmatization looks confusing.

###   - With lemmatization

In [89]:
# print out the top 10 words in each topic
tops_2 = ldamodel_2.print_topics(num_topics=20, num_words=10)

In [92]:
tbl_ec2 = pd.DataFrame([re.findall("\"\w*[.']?\w*\"", x[1]) for x in tops_2])

In [93]:
tbl_ec2

,0,1,2,3,4,5,6,7,8,9
0,"""call""","""rude""","""staff""","""doctor""","""office""","""never""","""worst""","""return""","""ever""","""phone"""
1,"""surgery""","""dr.""","""surgeon""","""recommend""","""perform""","""would""","""dr""","""result""","""highly""","""knee"""
2,"""tell""","""go""","""get""","""say""","""doctor""","""office""","""call""","""n't""","""would""","""see"""
3,"""family""","""friend""","""dr.""","""recommend""","""schwartz""","""member""","""refer""","""satisfy""","""rat""","""many"""
4,"""husband""","""cancer""","""mother""","""hospital""","""life""","""day""","""'s""","""implant""","""teeth""","""u"""
5,"""doctor""","""best""","""dr""","""would""","""recommend""","""ever""","""dr.""","""care""","""life""","""anyone"""
6,"""n't""","""'s""","""go""","""say""","""doctor""","""know""","""want""","""like""","""get""","""could"""
7,"""pain""","""back""","""problem""","""help""","""dr.""","""foot""","""year""","""month""","""get""","""severe"""
8,"""care""","""patient""","""dr.""","""doctor""","""physician""","""excellent""","""best""","""knowledgeable""","""'s""","""medical"""
9,"""surgery""","""procedure""","""go""","""dr.""","""breast""","""would""","""result""","""look""","""dentist""","""do"""


> With more topics, there are still topics that are hard to label, like topic 6. Moreover, there are topics (e.g. Topic 3 and Topic 17) that seem to be overlapping and hard to differentiate.

In [94]:
with open('analysis.txt', 'a') as f:
    f.writelines("\nExtra Credit\n\nTopics without lemmatization:\n")

In [95]:
tbl_ec1_ori.to_csv(r'analysis.txt', index=None, sep=' ', mode='a')

In [96]:
with open('analysis.txt', 'a') as f:
    f.writelines('All of the topics without lemmatization looks confusing.\n')
    f.writelines("\nTopics with lemmatization:\n")

In [97]:
tbl_ec2.to_csv(r'analysis.txt', index=None, sep=' ', mode='a')

In [98]:
with open('analysis.txt', 'a') as f:
    f.writelines("With more topics, there are still topics that are hard to label, like topic 6. Moreover, there are topics (e.g. Topic 3 and Topic 17) that seem to be overlapping and hard to differentiate.\n")